## Importing libraries

In [3]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import shutil

## Preprocessing The Images

In [16]:
# Define directories
TRAINING_DIR = "Datasets/Training"
VALIDATION_DIR = "Datasets/Validation"

# Define the data generators
training_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=30, height_shift_range=0.2, fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow images from directories
train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 980 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


## Model 

In [17]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(256, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(384, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['acc'])

model.summary()

C:\Users\Rakeem Shaikh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 22, 22, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 6, 6, 384)      │     2,457,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 2, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 2048)           │     3,147,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,355,586 (31.87 MB)

 Trainable params: 8,355,586 (31.87 MB)

 Non-trainable params: 0 (0.00 B)

## Training

In [12]:
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [18]:
val_batch = next(iter(validation_generator))
print(val_batch)

(array([[[[0.94117653, 0.9333334 , 0.8352942 ],
         [0.90196085, 0.8588236 , 0.78823537],
         [0.87843144, 0.8313726 , 0.7843138 ],
         ...,
         [0.43921572, 0.40784317, 0.40000004],
         [0.44705886, 0.4156863 , 0.40784317],
         [0.43529415, 0.4039216 , 0.39607847]],

        [[0.97647065, 0.9607844 , 0.86274517],
         [0.9333334 , 0.882353  , 0.81568635],
         [0.8862746 , 0.83921576, 0.77647066],
         ...,
         [0.44705886, 0.4156863 , 0.40784317],
         [0.44705886, 0.4156863 , 0.40784317],
         [0.4431373 , 0.41176474, 0.4039216 ]],

        [[0.9843138 , 0.9686275 , 0.8705883 ],
         [0.9568628 , 0.909804  , 0.8235295 ],
         [0.91372555, 0.86274517, 0.78823537],
         ...,
         [0.454902  , 0.42352945, 0.4156863 ],
         [0.454902  , 0.42352945, 0.4156863 ],
         [0.44705886, 0.4156863 , 0.40784317]],

        ...,

        [[0.20392159, 0.28235295, 0.27450982],
         [0.14901961, 0.227451  , 0.21960786

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50
)

Epoch 1/50


C:\Users\Rakeem Shaikh\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - acc: 0.7199 - loss: 0.5595
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.7998 - loss: 0.4086
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.8517 - loss: 0.3937
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.9053 - loss: 0.2430
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.9222 - loss: 0.1957
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 187us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.9052 - loss: 0.2763
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - acc: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - acc: 0.9065 - loss: 0.2350
Epoch 14/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - acc: 0.0000e+00 - loss: 0.0000e+0

In [20]:
# Save the entire model
model.save('fire_detection_model.h5')

## Predictions

In [27]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


# Load and preprocess the image
img_path = 'Datasets/predict/fireimg2.jpeg'  # Replace with your image path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Define class names
class_names = ['fire', 'no_fire']

# Output the prediction
print("Predicted class:", class_names[predicted_class[0]])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted class: fire
